In [10]:
import reset
from pathlib import Path
import numpy as np
import tifffile
import pandas as pd

In [14]:
# Assign variables from reset
animal_loc = reset.animal_loc
animal_id = reset.animal_id
hemi = reset.hemi
divisor = reset.divisor
down_x = reset.down_x
down_y = reset.down_y
loc_max = reset.loc_max
grid = reset.grid
frame_len = reset.frame_len

In [15]:
# Define paths for input and output images
image_loc = animal_loc / "concatenated" / hemi
save_loc = animal_loc / "trimmed" / hemi

# Ensure save location exists
save_loc.mkdir(parents=True, exist_ok=True)

In [16]:
# Initialize DataFrame for tracking
df = pd.DataFrame(columns=["Location", "Initial Length", "First Frame", "Last Frame"])

In [ ]:
# Loop over each location
for loc in range(1, loc_max + 1):
    # Construct file paths
    image_name = f"{animal_id}_{hemi}_{loc}.tif"
    image_path = image_loc / image_name
    save_path = save_loc / image_name

    # Load image
    image_array = tifffile.imread(image_path)
    xsize = int(down_x / grid)
    ysize = int(down_y / grid)

    # Initialize comparison sets
    old_set = set()
    start_trim = False
    f1 = None  # Start frame for trimming

    # Compare consecutive frames to detect trimming start
    for i in range(len(image_array) - 1):
        diff_md_list = []
        
        # Extract consecutive frames
        tdframe1 = image_array[i]
        tdframe2 = image_array[i + 1]

        # Compute median differences within grid blocks
        for y in range(grid):
            for x in range(grid):
                rect1 = tdframe1[y * ysize:(y + 1) * ysize, x * xsize:(x + 1) * xsize]
                rect2 = tdframe2[y * ysize:(y + 1) * ysize, x * xsize:(x + 1) * xsize]
                diff_md = np.median(np.abs(rect1 - rect2).flatten())
                diff_md_list.append(diff_md)

        # Compare sets of median differences
        new_set = set(diff_md_list)

        # Detect trimming start
        if i > 0 and old_set != new_set:
            start_trim = True
            f1 = i + 1  # Record start frame
            break

        # Update old set
        old_set = new_set

    # Perform trimming if necessary
    if start_trim and f1 is not None:
        last_frame = min(f1 + frame_len - 1, len(image_array) - 1)

        # Identify frames to delete
        delf_list = list(range(f1 - 1)) + list(range(last_frame + 1, len(image_array)))

        # Trim and save image array
        new_image_array = np.delete(image_array, delf_list, axis=0)
        tifffile.imwrite(save_path, new_image_array)
    else:
        print(f"Less than {frame_len} frames available to trim or no start found for location {loc}.")

    # Update DataFrame with trim information
    df.loc[len(df.index)] = [str(loc), len(image_array), f1, last_frame if start_trim else "N/A"]

In [18]:
# Save DataFrame to Excel
excel_path = save_loc / f"{animal_id}_{hemi}_trim.xlsx"
df.to_excel(excel_path, index=False)
print(f"Trimming info saved to {excel_path}")

Trimming info saved to /Volumes/BaffaloSSDPUTU3C1TB/rbak_data/rbak006/trimmed/l/rbak006_l_trim.xlsx


In [19]:
# Verify Excel contents
print(pd.read_excel(excel_path))

    Location  Initial Length  First Frame  Last Frame
0          1              67            3          64
1          2              66            4          65
2          3              66            3          64
3          4              66            2          63
4          5              68            4          65
5          6              69            4          65
6          7              65            4          64
7          8              75            4          65
8          9              69            5          66
9         10              65            4          64
10        11              69            3          64
11        12              67            4          65
12        13              65            3          64
13        14              66            5          65
14        15              67           20          66
15        16              68            5          66
16        17              66            4          65
17        18              66